In [ ]:
pip install flair

In [2]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install numpy==1.24.3

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install langdetect pymupdf

Note: you may need to restart the kernel to use updated packages.


In [1]:
import re
import fitz  # PyMuPDF
from langdetect import detect
from flair.embeddings import TransformerDocumentEmbeddings
from flair.data import Sentence
from flair.embeddings import TransformerDocumentEmbeddings
from flair.data import Sentence
import numpy as np
import torch

sentence = Sentence("This is a test.")
embedding_model = TransformerDocumentEmbeddings('bert-base-uncased')
embedding_model.embed(sentence)
PDF_PATH = "C:/Users/OUMAIMA/Downloads/cv_oumeymaFinal (1).pdf"
# Charger un modèle transformer de Flair (comme BERT)
embedding_model = TransformerDocumentEmbeddings('bert-base-uncased')

def extract_text_from_pdf(pdf_path):
    """Extract text from PDF using PyMuPDF."""
    text = ""
    doc = fitz.open(pdf_path)
    for page in doc:
        page_text = page.get_text()
        if page_text.strip():
            text += page_text + "\n"
    doc.close()
    return text

def clean_text(text):
    """Clean text by replacing accented characters and removing special chars except \n and spaces."""
    text = text.replace('à', 'a').replace('â', 'a').replace('ä', 'a')
    text = text.replace('é', 'e').replace('è', 'e').replace('ê', 'e').replace('ë', 'e')
    text = text.replace('î', 'i').replace('ï', 'i')
    text = text.replace('ô', 'o').replace('ö', 'o')
    text = text.replace('ù', 'u').replace('û', 'u').replace('ü', 'u')
    text = text.replace('ç', 'c')
    text = re.sub(r'[^\w\s\n]', '', text)
    return text

def detect_language(text):
    """Detect if the text is English or French with a larger sample and header check."""
    try:
        sample = text[:1000].lower()
        lang = detect(sample)
        if re.search(r'\b(formation|éducation|compétences|projets académiques)\b', sample):
            return "French"
        return "French" if lang == "fr" else "English" if lang == "en" else f"Other ({lang})"
    except:
        return "Unknown"

def mock_flair_similarity(text1, text2):
    """Placeholder for flair similarity; use keyword overlap as a proxy with section-specific keywords."""
    # Define keywords for each section type
    section_keywords = {
        "experience": ["worked", "developed", "experience", "team", "project", "intern", "built"],
        "skills": ["skills", "languages", "frameworks", "proficient", "tools", "databases"],
        "education": ["education", "degree", "graduated", "school", "university", "engineering"]
    }
    # Use the appropriate keyword set based on section_type
    keywords = section_keywords.get(section_type, section_keywords["experience"])
    text1_words = text1.lower().split()
    text2_words = text2.lower().split()
    common_words = set(text1_words) & set(text2_words) & set(keywords)
    score = len(common_words) / len(keywords) if keywords else 0.0  # Normalized score [0, 1]
    print(f"Text1 words: {text1_words[:5]}... Text2 words: {text2_words[:5]}... Common: {common_words} (Keywords: {keywords})")
    return score
def flair_similarity(text1, text2):
    """Compare deux textes en utilisant des embeddings contextuels Flair."""
    sentence1 = Sentence(text1)
    sentence2 = Sentence(text2)

    # Générer les embeddings
    embedding_model.embed(sentence1)
    embedding_model.embed(sentence2)

    # Extraire les vecteurs
    vec1 = sentence1.embedding.detach().numpy()
    vec2 = sentence2.embedding.detach().numpy()

    # Cosine similarity entre les vecteurs
    score = np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

    return float(score)  # score entre -1 et 1
def find_best_section_block(lines, standard_description, section_type="experience"):
    """Find the best block for a given section (experience, skills, or education) by sliding a 5-line window and optimizing."""
    if len(lines) < 5:
        return "CV too short for 5-line window."

    # Standard description for the specific section
    standard_desc = standard_description.lower()

    # Initialize sliding window
    window_size = 5
    best_start = 0
    best_end = window_size
    max_score = 0.0
    current_start = 0

    # Slide 5-line window across CV with debugging
    print(f"\n=== SLIDING WINDOW PROCESS ({section_type.capitalize()}) ===")
    while current_start <= len(lines) - window_size:
        window = "\n".join(lines[current_start:current_start + window_size])
        score = flair_similarity(window, standard_desc)
        print(f"Window {current_start}-{current_start + window_size}: {window[:50]}... (Score: {score:.2f})")
        if score > max_score:
            max_score = score
            best_start = current_start
            best_end = current_start + window_size
            print(f"  -> New best window found! Score: {max_score:.2f}")
        current_start += 1

    # Optimize window size around best 5-line block
    best_block = "\n".join(lines[best_start:best_end])
    current_block = best_block
    current_score = max_score
    min_lines = 1
    max_lines = len(lines)

    print(f"\n=== WINDOW OPTIMIZATION ({section_type.capitalize()}) ===")
    print(f"Initial best block ({best_end - best_start} lines):\n{best_block}")

    # Expand or contract to find optimal length
    while True:
        # Try adding a line
        if best_end < len(lines):
            new_end = best_end + 1
            new_block = "\n".join(lines[best_start:new_end])
            new_score = flair_similarity(new_block, standard_desc)
            print(f"Trying +1 line ({new_end - best_start} lines): {new_block[:50]}... (Score: {new_score:.2f})")
            if new_score > current_score:
                current_block = new_block
                current_score = new_score
                best_end = new_end
                print(f"  -> Improved! New score: {current_score:.2f}")
            else:
                print(f"  -> No improvement, stopping expansion.")
                break
        # Try removing a line from start if more than 5 lines
        elif best_start < best_end - 1:
            new_start = best_start + 1
            new_block = "\n".join(lines[new_start:best_end])
            new_score = flair_similarity(new_block, standard_desc)
            print(f"Trying -1 line ({best_end - new_start} lines): {new_block[:50]}... (Score: {new_score:.2f})")
            if new_score >= current_score:
                current_block = new_block
                current_score = new_score
                best_start = new_start
                print(f"  -> Improved! New score: {current_score:.2f}")
            else:
                print(f"  -> No improvement, stopping contraction.")
                break
        else:
            break

    return current_block, best_end - best_start

# MAIN
raw_text = extract_text_from_pdf(PDF_PATH)
cleaned_text = clean_text(raw_text)
lines = cleaned_text.splitlines()  # Split into lines here
language = detect_language(raw_text)
print(f"\n📄 Langue détectée : {language}")

# Standard descriptions for each section
standard_experience = "Completed a 4-month internship at TechSolutions where I developed a RESTful API using Node.js and MongoDB, improving data processing."
standard_skills = "Proficient in Python, Java, JavaScript, and frameworks like Angular, with experience in databases such as MySQL and Firebase."
standard_education = "Graduated with a Bachelor’s Degree in Computer Science from a university in 2023."

# Detect blocks for each section
best_experience_block, experience_line_count = find_best_section_block(lines, standard_experience, "experience")
best_skills_block, skills_line_count = find_best_section_block(lines, standard_skills, "skills")
best_education_block, education_line_count = find_best_section_block(lines, standard_education, "education")

# Print results
print("\n=== SECTION D'EXPÉRIENCE DÉTECTÉE ===")
print(f"Best experience block:\n{best_experience_block}")
print(f"Number of lines: {experience_line_count}")

print("\n=== SECTION DES COMPÉTENCES DÉTECTÉE ===")
print(f"Best skills block:\n{best_skills_block}")
print(f"Number of lines: {skills_line_count}")

print("\n=== SECTION D'ÉDUCATION DÉTECTÉE ===")
print(f"Best education block:\n{best_education_block}")
print(f"Number of lines: {education_line_count}")

ImportError: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.11 from "C:\Users\OUMAIMA\anaconda3\python.exe"
  * The NumPy version is: "1.24.3"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: DLL load failed while importing _multiarray_umath: Le module spécifié est introuvable.
